<h1>Segmentation &amp; Clustering Neighborhoods Assignment Submission</h1>
<h2>Part II</h2>


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# Assign the wikipedia URL for the Canadian postal codes site for all codes starting with the letter "M".
# to the newly named variable "url"
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text
soup=BeautifulSoup(source,'lxml')

# Using the BeautifulSoup package for web scraping.

In [4]:
# Using the BeautifulSoup package for more complicated cases of web scraping.
htmltable=soup.find('table',{'class' : 'wikitable sortable' })
print(htmltable.prettify())


<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [5]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows


list_table = tableDataText(htmltable)


# Transform the data into a pandas dataframe

In [6]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(list_table[1:], columns=list_table[0])
df.columns=[ 'PostalCode', 'Borough', 'Neighborhood']

In [7]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [8]:
# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
# separated with a comma as shown in row 11 in the above table.
df = df.groupby(['PostalCode'],as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, 
# the value of the Borough and the Neighborhood columns will be Queen's Park.
df.loc[(df['Neighborhood']== 'Not assigned'), 'Neighborhood'] = df['Borough']
df.head()
print(df.loc[df['PostalCode'] == 'M5A'])
print(df.loc[df['PostalCode'] == 'M7A'])

   PostalCode           Borough               Neighborhood
53        M5A  Downtown Toronto  Harbourfront, Regent Park
   PostalCode       Borough  Neighborhood
85        M7A  Queen's Park  Queen's Park


In [10]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print(df.shape)

(103, 3)


# Get the latitude and the longitude coordinates of each neighborhood.

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, 
in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and 
the longitude coordinates of each neighborhood. However, recently Google started charging for their API:
http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, 
so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates 
of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code 
and the result would be None, and then make the call again and you would get the coordinates. 
So, in order to make sure that you get the coordinates for all of our neighborhoods, 
you can run a while loop for each postal code. Taking postal code M5G as an example, 
your code would look something like this:
import geocoder # import geocoder

#initialize your variable to None
lat_lng_coords = None
postal_code='M5G'
loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

  latitude = lat_lng_coords[0]
  longitude = lat_lng_coords[1]

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates 
of the neighborhoods using the Geocoder package, here is a link to a csv file that has the 
geographical coordinates of each postal code: http://cocl.us/Geospatial_data.

# Use Geospatial csv file to create the following dataframe:

In [21]:
import wget
geo_url='http://cocl.us/Geospatial_data'
geospatial_data = wget.download(geo_url)
df_geospatial = pd.read_csv(geospatial_data)
df_geospatial.columns=['PostalCode','Latitude','Longitude']
df_geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df_merge=pd.merge(df,df_geospatial,on='PostalCode')

df_merge.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [27]:
df_merge.loc[df_merge['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


# Explore and cluster the neighborhoods in Toronto. 
# We can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.
